In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [8]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-a-30/RFM/RFM_ht_data.csv')
df.columns = df.columns.str.lower()
df.fillna(0, inplace=True)
df['invoicedate'] = pd.to_datetime(df['invoicedate'])
df['invoiceno'] = df['invoiceno'].astype(str)
df['customercode'] = df['customercode'].astype(str)
df.head()

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,invoiceno,customercode,invoicedate,amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [9]:
df['invoicedate'].describe()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                  332730
unique                     30
top       2020-09-30 00:00:00
freq                    19781
first     2020-09-01 00:00:00
last      2020-09-30 00:00:00
Name: invoicedate, dtype: object

In [10]:
grouped = df.groupby('customercode')['invoiceno'].count()
result = grouped.sort_values(ascending=False)
result

customercode
19057820    204
13215452    113
13032521    106
19080880     99
99003061     90
           ... 
18053019      1
18053055      1
18053072      1
18053075      1
02213019      1
Name: invoiceno, Length: 123733, dtype: int64

In [11]:
last_date = df['invoicedate'].max()
last_date

Timestamp('2020-09-30 00:00:00')

In [13]:
rfmTable = df.groupby('customercode').agg({'invoicedate': lambda x: (last_date - x.max()).days,
                                               'invoiceno': lambda x: len(x),
                                               'amount' : lambda x: x.sum()})

rfmTable['invoicedate'] = rfmTable['invoicedate'].astype(int)
rfmTable.rename(columns={'invoicedate' : 'recency',
                         'invoiceno' : 'frequency',
                         'amount' : 'monetary_value'}, inplace=True)

In [15]:
rfmTable.head()

,recency,frequency,monetary_value
customercode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [17]:
rfmTable.shape[0]

123733

In [20]:
df['customercode'].nunique()

123733

In [26]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [27]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [28]:
rfmSegmentation = rfmTable

In [29]:
def RClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]:
        return 3
    else:
        return 4
    
def FMClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]:
        return 2
    else:
        return 1
    
    
    

In [30]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str) \




In [32]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customercode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


In [34]:
m_segment_class4 = rfmSegmentation[(rfmSegmentation['M_Quartile'] == 4) & (rfmSegmentation['RFMClass'].str.contains('4'))]
upper_bound = m_segment_class4['monetary_value'].max()
upper_bound


765.0

In [36]:
f_segment_class1 = rfmSegmentation[(rfmSegmentation['F_Quartile'] == 1) & (rfmSegmentation['RFMClass'].str.contains('1'))]
lower_bound = f_segment_class1['frequency'].min()
lower_bound

4

In [39]:
r_segment_class2 = rfmSegmentation[(rfmSegmentation['R_Quartile'] == 2) & (rfmSegmentation['RFMClass'].str.contains('2'))]
max_days_since_last_purchase = r_segment_class2['recency'].max()
max_days_since_last_purchase

8

In [41]:
segment_111 = rfmSegmentation[rfmSegmentation['RFMClass'] == '111']
num_users = len(segment_111)
num_users


9705

In [43]:
segment_311 = rfmSegmentation[rfmSegmentation['RFMClass'] == '311']
num_users = len(segment_311)
num_users

1609

In [46]:
largest_segment = rfmSegmentation['RFMClass'].value_counts().idxmax()
largest_segment

'444'

In [47]:
small_segment = rfmSegmentation['RFMClass'].value_counts().idxmin()
small_segment

'414'

In [50]:
smallest_segment = rfmSegmentation['RFMClass'].value_counts().idxmin()
num_users_smallest_segment = len(rfmSegmentation[rfmSegmentation['RFMClass'] == smallest_segment])
num_users_smallest_segment


2